# Preliminary Analysis

This notebook contains a preliminary performance analysis of the two proposed pseudonymization solutions.

The performance data was generated by executing the `uc-runner.sh` for both solutions (i.e., `webid-webid` and `webid-didkey`)
using the following parameters:
- Nr. of iterations: `-n=100` per solution.
- Caching behavior of the documentloader: `-d=0`, which means that the document loaders apply no caching at all. The caching configurations can be found in `src/profiling/config.ts`.


Imports.

In [1]:
import pandas as pd
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns

Helpers.

In [2]:
def read_report(path: str) -> pd.DataFrame:
    data = json.load(open(path, 'r'))
    df = pd.DataFrame(data['records'])
    df['path'] = path
    return df

def add_filepath_information(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    # basename
    df_out['basename'] = df_out.path.apply(os.path.basename)
    # parent dir
    df_out['parent_dir'] = df_out.path.apply(os.path.dirname).apply(os.path.basename)
    # grandparent dir
    df_out['grandparent_dir'] = df_out.path.apply(os.path.dirname).apply(os.path.dirname).apply(os.path.basename)
    return df_out

Global variables.

In [3]:
dir_experiment = 'n-100-dclo-0'
solution_timestamps = {
    '1732405225570': 'webid-didkey',
    '1732405664773': 'webid-webid'
}
fpaths_reports = sorted(glob(os.path.join(dir_experiment, '*', '*.json')))
len(fpaths_reports)

200

Creating a DataFrame from the performance reports.

In [4]:
dfs = pd.concat(map(read_report,  fpaths_reports))
dfs['experiment_tag'] = dfs.path.str.split('/').apply(lambda x: str(x[0]))
dfs['solution_tag'] = dfs.path.str.split('/').apply(lambda x: str(x[1])).apply(solution_timestamps.__getitem__)

print(dfs.shape)
dfs.head(3)

(2400, 11)


,index,name,start,end,delta,output,tag,className,path,experiment_tag,solution_tag
0,0,createDiplomaCredential,1732405227837,1732405227837,0,{'@context': ['https://www.w3.org/2018/credent...,university,SolidVCActor,n-100-dclo-0/1732405225570/multiactor-report-1...,n-100-dclo-0,webid-didkey
1,1,signDiplomaCredential,1732405227837,1732405227880,43,{'@context': ['https://www.w3.org/2018/credent...,university,SolidVCActor,n-100-dclo-0/1732405225570/multiactor-report-1...,n-100-dclo-0,webid-didkey
2,2,createIdentityCredential,1732405227880,1732405227880,0,{'@context': ['https://www.w3.org/2018/credent...,government,SolidVCActor,n-100-dclo-0/1732405225570/multiactor-report-1...,n-100-dclo-0,webid-didkey


Compute the nr. of records per solution, and ensure that the same amount of records have been collected for both solutions.

In [5]:
n_records_per_solution = dfs.solution_tag.value_counts(dropna=False).to_dict()
print('Nr. of records per solution', n_records_per_solution)
# Ensure that the same amount of records were registered for both solutions
assert n_records_per_solution['webid-webid'] == n_records_per_solution['webid-didkey']

Nr. of records per solution {'webid-didkey': 1200, 'webid-webid': 1200}


Compute descriptive statistics per experiment (`experiment_tag`),
solution (`solution_tag`),
actor (`tag`),
step name (`name`).

In [6]:
df_agg = dfs.groupby(['experiment_tag','solution_tag','tag','name',])['delta'].agg(['mean','std','var','count'])
df_agg = df_agg.unstack('solution_tag').swaplevel(0,1,axis=1).sort_index(axis=1)
df_agg = df_agg.round(2)
df_agg

solution_tag                                       webid-didkey           \
                                                          count     mean   
experiment_tag tag        name                                             
n-100-dclo-0   alice      createPresentation01              100     0.00   
                          createPresentation02              100    43.27   
                          deriveDiplomaCredential           100  1863.46   
                          deriveIdentityCredential          100  1877.86   
                          signPresentation01                100    33.28   
                          signPresentation02                100    42.24   
               government createIdentityCredential          100     0.00   
                          signIdentityCredential            100    17.49   
               recruiter  verifyPresentation01              100    44.35   
                          verifyPresentation02              100   122.37   
               university createDiplomaCredential           100     0.00   
                          signDiplomaCredential             100    16.21   

solution_tag                                                          \
                                                       std       var   
experiment_tag tag        name                                         
n-100-dclo-0   alice      createPresentation01        0.00      0.00   
                          createPresentation02       20.94    438.38   
                          deriveDiplomaCredential   103.97  10809.24   
                          deriveIdentityCredential   99.24   9849.35   
                          signPresentation01          8.11     65.76   
                          signPresentation02         10.13    102.57   
               government createIdentityCredential    0.00      0.00   
                          signIdentityCredential      2.54      6.45   
               recruiter  verifyPresentation01        5.11     26.11   
                          verifyPresentation02       42.89   1839.49   
               university createDiplomaCredential     0.00      0.00   
                          signDiplomaCredential       5.41     29.28   

solution_tag                                       webid-webid           \
                                                         count     mean   
experiment_tag tag        name                                            
n-100-dclo-0   alice      createPresentation01             100     0.05   
                          createPresentation02             100    42.78   
                          deriveDiplomaCredential          100  1865.78   
                          deriveIdentityCredential         100  1892.26   
                          signPresentation01               100    33.09   
                          signPresentation02               100    40.93   
               government createIdentityCredential         100     0.01   
                          signIdentityCredential           100    17.15   
               recruiter  verifyPresentation01             100    91.15   
                          verifyPresentation02             100   118.30   
               university createDiplomaCredential          100     0.00   
                          signDiplomaCredential            100    16.01   

solution_tag                                                          
                                                       std       var  
experiment_tag tag        name                                        
n-100-dclo-0   alice      createPresentation01        0.22      0.05  
                          createPresentation02       10.00     99.93  
                          deriveDiplomaCredential    97.32   9471.43  
                          deriveIdentityCredential  118.49  14040.48  
                          signPresentation01          9.42     88.65  
                          signPresentation02          3.56     12.69  
             

In [7]:
s1 = 'webid-webid'
s2 = 'webid-didkey'

actor = 'recruiter'
step_name = 'verifyPresentation01'
metric = 'mean'

In [8]:
df_agg

solution_tag                                       webid-didkey           \
                                                          count     mean   
experiment_tag tag        name                                             
n-100-dclo-0   alice      createPresentation01              100     0.00   
                          createPresentation02              100    43.27   
                          deriveDiplomaCredential           100  1863.46   
                          deriveIdentityCredential          100  1877.86   
                          signPresentation01                100    33.28   
                          signPresentation02                100    42.24   
               government createIdentityCredential          100     0.00   
                          signIdentityCredential            100    17.49   
               recruiter  verifyPresentation01              100    44.35   
                          verifyPresentation02              100   122.37   
               university createDiplomaCredential           100     0.00   
                          signDiplomaCredential             100    16.21   

solution_tag                                                          \
                                                       std       var   
experiment_tag tag        name                                         
n-100-dclo-0   alice      createPresentation01        0.00      0.00   
                          createPresentation02       20.94    438.38   
                          deriveDiplomaCredential   103.97  10809.24   
                          deriveIdentityCredential   99.24   9849.35   
                          signPresentation01          8.11     65.76   
                          signPresentation02         10.13    102.57   
               government createIdentityCredential    0.00      0.00   
                          signIdentityCredential      2.54      6.45   
               recruiter  verifyPresentation01        5.11     26.11   
                          verifyPresentation02       42.89   1839.49   
               university createDiplomaCredential     0.00      0.00   
                          signDiplomaCredential       5.41     29.28   

solution_tag                                       webid-webid           \
                                                         count     mean   
experiment_tag tag        name                                            
n-100-dclo-0   alice      createPresentation01             100     0.05   
                          createPresentation02             100    42.78   
                          deriveDiplomaCredential          100  1865.78   
                          deriveIdentityCredential         100  1892.26   
                          signPresentation01               100    33.09   
                          signPresentation02               100    40.93   
               government createIdentityCredential         100     0.01   
                          signIdentityCredential           100    17.15   
               recruiter  verifyPresentation01             100    91.15   
                          verifyPresentation02             100   118.30   
               university createDiplomaCredential          100     0.00   
                          signDiplomaCredential            100    16.01   

solution_tag                                                          
                                                       std       var  
experiment_tag tag        name                                        
n-100-dclo-0   alice      createPresentation01        0.22      0.05  
                          createPresentation02       10.00     99.93  
                          deriveDiplomaCredential    97.32   9471.43  
                          deriveIdentityCredential  118.49  14040.48  
                          signPresentation01          9.42     88.65  
                          signPresentation02          3.56     12.69  
             

In [9]:
df_agg_mean = df_agg.loc(axis=1)[:, 'mean'].loc[dir_experiment, ]
df_agg_mean

solution_tag                        webid-didkey webid-webid
                                            mean        mean
tag        name                                             
alice      createPresentation01             0.00        0.05
           createPresentation02            43.27       42.78
           deriveDiplomaCredential       1863.46     1865.78
           deriveIdentityCredential      1877.86     1892.26
           signPresentation01              33.28       33.09
           signPresentation02              42.24       40.93
government createIdentityCredential         0.00        0.01
           signIdentityCredential          17.49       17.15
recruiter  verifyPresentation01            44.35       91.15
           verifyPresentation02           122.37      118.30
university createDiplomaCredential          0.00        0.00
           signDiplomaCredential           16.21       16.01